In [17]:
#!/usr/bin/env sage

# Import necessary SageMath functionality
from sage.groups.perm_gps.permgroup import PermutationGroup
# DirectProductGroup is in a different module, we'll use alternative methods if needed
# from sage.groups.perm_gps.permgroup_named import DirectProductGroup
from sage.libs.gap.libgap import libgap
from itertools import product

# Define a helper function to print group information
def analyze_group(G, name):
    print(f"\n{'='*60}")
    print(f"Analysis of {name}")
    print(f"{'='*60}")
    print(f"Order: {G.order()}")
    try:
        print(f"Group structure description: {G.structure_description()}")
    except Exception as e:
        print(f"Structure description unavailable: {e}")
    
    # Attempt to get the ID in the Small Groups library
    try:
        id_group = G.gap().IdGroup()
        print(f"Small Groups ID: {id_group}")
    except Exception as e:
        print(f"Unable to determine Small Groups ID: {e}")
    
    # List some basic properties
    print(f"Is abelian: {G.is_abelian()}")
    print(f"Is nilpotent: {G.is_nilpotent()}")
    print(f"Is solvable: {G.is_solvable()}")
    
    # Get conjugacy classes info if the group is small enough
    if G.order() <= 64:
        try:
            cc = G.conjugacy_classes()
            print(f"Number of conjugacy classes: {len(cc)}")
        except Exception as e:
            print(f"Could not compute conjugacy classes: {e}")
    
    # For the generators, print their orders
    print("\nGenerator orders:")
    for gen in G.gens():
        try:
            # Fix: Use the proper method to compute element order
            order = G.order_from_element(gen)
            print(f"{gen} has order {order}")
        except AttributeError:
            # Alternative method if order_from_element doesn't exist
            try:
                order = gen.order()
                print(f"{gen} has order {order}")
            except:
                # Last resort
                print(f"{gen} - order could not be determined")

# Part 1: Define Ginternal - Internal Symmetry Group (order 32)
# We need to define permutation representations for each generator:
# c1, d1, c2, d2, S

# Let's represent the state of each helix with coordinates:
# (helix_id, chirality, orientation) where:
# - helix_id = 1 or 2
# - chirality = 0 (right-handed) or 1 (left-handed)
# - orientation = 0 (up) or 1 (down)

# This gives us 8 possible states, which we'll map to numbers 1-8:
# 1: (1, 0, 0) = Helix 1, right-handed, up
# 2: (1, 0, 1) = Helix 1, right-handed, down
# 3: (1, 1, 0) = Helix 1, left-handed, up
# 4: (1, 1, 1) = Helix 1, left-handed, down
# 5: (2, 0, 0) = Helix 2, right-handed, up
# 6: (2, 0, 1) = Helix 2, right-handed, down
# 7: (2, 1, 0) = Helix 2, left-handed, up
# 8: (2, 1, 1) = Helix 2, left-handed, down

# Now define the generator permutations based on their actions:

# c1: Swaps chirality of Helix 1 (maps 1↔3, 2↔4, leaves 5-8 fixed)
c1 = [(1,3), (2,4)]

# d1: Flips orientation of Helix 1 (maps 1↔2, 3↔4, leaves 5-8 fixed)
d1 = [(1,2), (3,4)]

# c2: Swaps chirality of Helix 2 (maps 5↔7, 6↔8, leaves 1-4 fixed)
c2 = [(5,7), (6,8)]

# d2: Flips orientation of Helix 2 (maps 5↔6, 7↔8, leaves 1-4 fixed)
d2 = [(5,6), (7,8)]

# S: Swaps identities of Helix 1 and Helix 2 (maps 1↔5, 2↔6, 3↔7, 4↔8)
S = [(1,5), (2,6), (3,7), (4,8)]

# Create the internal symmetry group
G_internal = PermutationGroup([c1, d1, c2, d2, S])

# Part 2: Define Gexternal_point_group - External Point Group (order 8)
# We need to define permutation representations for:
# R90: 90-degree rotation around z-axis (order 4)
# σh: reflection across xy-plane (order 2)

# For this, we need a different representation. Let's represent the 
# 8 positions around the z-axis (a, b, c, d, a', b', c', d')
# where unprimed positions are above the xy-plane and primed below.
# We'll map these to numbers 9-16:
# 9: a  (above xy-plane)
# 10: b (above xy-plane)
# 11: c (above xy-plane)
# 12: d (above xy-plane)
# 13: a' (below xy-plane)
# 14: b' (below xy-plane)
# 15: c' (below xy-plane)
# 16: d' (below xy-plane)

# R90: 90-degree rotation (a→b→c→d→a, a'→b'→c'→d'→a')
R90 = [(9,10,11,12), (13,14,15,16)]

# σh: reflection across xy-plane (a↔a', b↔b', c↔c', d↔d')
sigma_h = [(9,13), (10,14), (11,15), (12,16)]

# Create the external point group
G_external = PermutationGroup([R90, sigma_h])

# Part 3: Construct Gtotal as the direct product
# For a direct product of permutation groups acting on disjoint sets,
# we can simply combine their generators
G_total = PermutationGroup(G_internal.gens() + G_external.gens())

# Alternatively, if you have direct product functionality in your SageMath version:
# Method 1: Try using direct_product if available
try:
    G_total_alt1 = G_internal.direct_product(G_external)
    print("\nSuccessfully created G_total using direct_product method")
except Exception as e1:
    print(f"\nCould not create direct product using direct_product method: {e1}")
    
# Method 2: Try using CartesianProduct if available
try:
    from sage.categories.cartesian_product import cartesian_product
    G_total_alt2 = cartesian_product([G_internal, G_external])
    print("Successfully created G_total using cartesian_product method")
except Exception as e2:
    print(f"Could not create direct product using cartesian_product method: {e2}")
    
# Note: We're primarily using G_total (constructed from combined generators)
# for our analysis, as this works reliably across SageMath versions

# Part 4: Analyze all groups
analyze_group(G_internal, "G_internal (Internal Symmetry Group)")
analyze_group(G_external, "G_external (External Point Group)")
analyze_group(G_total, "G_total (Full Symmetry Group)")

# Verify that G_total has the expected structure
print("\n\nVerification of G_total Structure:")
print(f"Is G_total order = G_internal order × G_external order? {G_total.order() == G_internal.order() * G_external.order()}")

# Additional Analysis: Check the center of the groups
print("\n\nCenters of the Groups:")
Z_internal = G_internal.center()
Z_external = G_external.center()
Z_total = G_total.center()

print(f"Center of G_internal has order: {Z_internal.order()}")
print(f"Center of G_external has order: {Z_external.order()}")
print(f"Center of G_total has order: {Z_total.order()}")
# One more check: validate the isomorphism of G_internal with SmallGroup(32,27)
print("\nValidating G_internal Structure:")

# Method 1: Try using SageMath's id_group method
try:
    id_tuple = G_internal.id_group()
    print(f"G_internal has SageMath ID: {id_tuple}")
    
    if id_tuple == (32, 27):
        print("Confirmed: G_internal is isomorphic to SmallGroup(32,27)")
    else:
        print(f"G_internal is isomorphic to SmallGroup{id_tuple}")
except Exception as e:
    print(f"SageMath id_group method failed: {e}")

# Method 2: Try using GAP's IdGroup function via libgap interface
try:
    # Convert the permutation group to GAP format safely
    gap_group = G_internal.gap()
    # Use libgap's IdGroup function directly
    id_gap = libgap.IdGroup(gap_group)
    print(f"G_internal has GAP ID: {id_gap}")
    
    if str(id_gap) == "[ 32, 27 ]":
        print("Confirmed via GAP: G_internal is isomorphic to SmallGroup(32,27)")
    else:
        print(f"According to GAP, G_internal is SmallGroup{id_gap}")
except Exception as e:
    print(f"GAP IdGroup method failed: {e}")

# Method 3: Check key structural properties
try:
    print("\nStructural properties of G_internal:")
    print(f"Order: {G_internal.order()}")
    
    # Check if it's a semidirect product structure
    is_nilpotent = G_internal.is_nilpotent()
    is_abelian = G_internal.is_abelian()
    
    print(f"Is nilpotent: {is_nilpotent}")
    print(f"Is abelian: {is_abelian}")
    
    # Check center order (Z2^4 ⋊ Z2 should have center of order 4)
    center_order = G_internal.center().order()
    print(f"Center order: {center_order}")
    
    # Check if it has the right number of elements of each order
    elements_of_order_2 = sum(1 for g in G_internal if g != G_internal.identity() and g^2 == G_internal.identity())
    print(f"Elements of order 2: {elements_of_order_2}")
    
    print("Based on these properties, we can determine if G_internal matches the expected structure")
    print("Expected for SmallGroup(32,27): Order=32, non-abelian, nilpotent, center order=4")
except Exception as e:
    print(f"Structural analysis failed: {e}")


Successfully created G_total using direct_product method
Successfully created G_total using cartesian_product method

Analysis of G_internal (Internal Symmetry Group)
Order: 32
Group structure description: (C2 x C2 x C2 x C2) : C2
Small Groups ID: [ 32, 27 ]
Is abelian: False
Is nilpotent: True
Is solvable: True
Number of conjugacy classes: 14

Generator orders:
(5,6)(7,8) has order 2
(5,7)(6,8) has order 2
(1,2)(3,4) has order 2
(1,3)(2,4) has order 2
(1,5)(2,6)(3,7)(4,8) has order 2

Analysis of G_external (External Point Group)
Order: 8
Group structure description: C4 x C2
Small Groups ID: [ 8, 2 ]
Is abelian: True
Is nilpotent: True
Is solvable: True
Number of conjugacy classes: 8

Generator orders:
(9,10,11,12)(13,14,15,16) has order 4
(9,13)(10,14)(11,15)(12,16) has order 2

Analysis of G_total (Full Symmetry Group)
Order: 256
Group structure description: C2 x C4 x ((C2 x C2 x C2 x C2) : C2)
Small Groups ID: [ 256, 27015 ]
Is abelian: False
Is nilpotent: True
Is solvable: True

